# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [1]:
import main

In [69]:
min_length = 0
max_length = 1e6

qoutation =   f"""SELECT length, title 
                    FROM film 
                    WHERE length BETWEEN {min_length} AND {max_length}
                    GROUP BY length, title
                    ORDER BY length"""

main.pd.read_sql_query(qoutation, con=main.connection)

,length,title
0,46,Alien Center
1,46,Iron Moon
2,46,Kwai Homeward
3,46,Labyrinth League
4,46,Ridgemont Submarine
...,...,...
995,185,Muscle Bright
996,185,Pond Seattle
997,185,Soldiers Evolution
998,185,Sweet Brotherhood


In [100]:
qoutation =   f"""SELECT city, count(customer.first_name), count(customer.last_name)
                    FROM city 
                    JOIN address ON address.city_id = city.city_id
                    JOIN customer ON address.address_id = customer.address_id
                    WHERE city.city SIMILAR TO '%'
                    GROUP BY city.city
                    ORDER BY count(customer.first_name) desc"""

main.pd.read_sql_query(qoutation, con=main.connection)


,city,count,count
0,London,2,2
1,Aurora,2,2
2,Molodetno,1,1
3,Yerevan,1,1
4,Vitria de Santo Anto,1,1
...,...,...,...
592,Garden Grove,1,1
593,Novoterkassk,1,1
594,Asuncin,1,1
595,Maracabo,1,1


In [101]:
min_length = 0
max_length = 1e6

qoutation =   f"""SELECT film.length, avg(payment.amount)
                    FROM film
                    JOIN inventory ON inventory.film_id = film.film_id
                    JOIN rental ON rental.inventory_id = inventory.inventory_id  
                    JOIN payment ON payment.rental_id = rental.rental_id
                    WHERE film.length BETWEEN {min_length} AND {max_length}
                    GROUP BY film.length"""

a = main.pd.read_sql_query(qoutation, con=main.connection)

b = sum(a.avg)
c = a.size
d = b/c
print(d)

2.0781306160399717


In [105]:
quotation =   f"""SELECT category.name AS category, count(film.title)
            FROM film
            LEFT JOIN film_category ON film.film_id = film_category.film_id
            LEFT JOIN category ON film_category.category_id = category.category_id
            GROUP BY category.name"""

main.pd.read_sql_query(quotation, con=main.connection)

,category,count
0,Horror,56
1,Comedy,58
2,Sci-Fi,61
3,Drama,62
4,Foreign,73
5,Classics,57
6,Games,61
7,New,63
8,Travel,57
9,Music,51


In [124]:
qoutation =   f"""SELECT country.country as ccountry, count(customer.first_name)
                FROM city
                JOIN country ON city.country_id = country.country_id
                JOIN address ON address.city_id = city.city_id
                JOIN customer ON address.address_id = customer.address_id
                GROUP BY ccountry
                ORDER BY count(customer.first_name)"""

f = main.pd.read_sql_query(qoutation, con=main.connection)
f

,ccountry,count
0,Senegal,1
1,Estonia,1
2,New Zealand,1
3,Hungary,1
4,Sri Lanka,1
...,...,...
103,Mexico,30
104,Japan,31
105,United States,36
106,China,53


In [141]:
qoutation =   f"""SELECT count(customer.customer_id) as ccount, store.store_id
                FROM store
                JOIN customer ON customer.store_id = store.store_id
                GROUP BY store.store_id
                ORDER BY count(customer.customer_id)"""

f = main.pd.read_sql_query(qoutation, con=main.connection)
print(f)
f['ccount'].between(100, 300)

   ccount  store_id
0     273         2
1     326         1


0     True
1    False
Name: ccount, dtype: bool

In [142]:
main.client_by_sum_length(200)

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Wesley,Bull,4808
595,Clara,Shaw,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


In [149]:
qoutation =   f"""SELECT film.title, avg(payment.amount)
                    FROM film
                    JOIN inventory ON inventory.film_id = film.film_id
                    JOIN rental ON rental.inventory_id = inventory.inventory_id  
                    JOIN payment ON payment.rental_id = rental.rental_id
                    GROUP BY film.title"""


main.pd.read_sql_query(qoutation, con=main.connection)

,title,avg
0,Frontier Cabin,5.990000
1,Arachnophobia Rollercoaster,4.772609
2,Cruelty Unforgiven,1.156667
3,Intolerable Intentions,5.704286
4,Monsoon Cause,6.101111
...,...,...
953,Reef Salute,2.171818
954,Bugsy Song,5.101111
955,Orient Closer,4.754706
956,Brotherhood Blanket,4.294348


In [164]:
quotation =   f"""SELECT category.name AS category, count(film.title) as fcount
            FROM film
            LEFT JOIN film_category ON film.film_id = film_category.film_id
            LEFT JOIN category ON film_category.category_id = category.category_id
            GROUP BY category.name"""

a = main.pd.read_sql_query(quotation, con=main.connection)

quotation_2 =   f"""SELECT category.name AS category, sum(film.length) as fsum
            FROM film
            LEFT JOIN film_category ON film.film_id = film_category.film_id
            LEFT JOIN category ON film_category.category_id = category.category_id
            GROUP BY category.name
"""

b = main.pd.read_sql_query(quotation_2, con=main.connection)
b.fsum/a.fcount


0     112.482143
1     115.827586
2     108.196721
3     120.838710
4     121.698630
5     111.666667
6     127.836066
7     111.126984
8     113.315789
9     113.647059
10    111.609375
11    108.750000
12    128.202703
13    111.015152
14    114.782609
15    109.800000
dtype: float64

In [190]:
quotation =   f"""SELECT category.name AS category, max(film.length) as fmax
            FROM film
            LEFT JOIN film_category ON film.film_id = film_category.film_id
            LEFT JOIN category ON film_category.category_id = category.category_id
            GROUP BY category.name"""

a = main.pd.read_sql_query(quotation, con=main.connection)
a

,category,fmax
0,Horror,181
1,Comedy,185
2,Sci-Fi,185
3,Drama,181
4,Foreign,184
5,Classics,184
6,Games,185
7,New,183
8,Travel,185
9,Music,185


In [194]:
quotation =   f"""SELECT film.title, film.length
            FROM film
            ORDER BY film.length desc"""

a = main.pd.read_sql_query(quotation, con=main.connection)
a

,title,length
0,Muscle Bright,185
1,Control Anthem,185
2,Sweet Brotherhood,185
3,Pond Seattle,185
4,Chicago North,185
...,...,...
995,Kwai Homeward,46
996,Ridgemont Submarine,46
997,Alien Center,46
998,Labyrinth League,46
